# 1. Introdução

### 1.1. TLDR

* **Dashboard**:
    - https://lookerstudio.google.com/reporting/3e2706fd-dd91-4713-b904-1337024950a0

* **Fontes**:

  - Casos pela Universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports))
  - Vacinação pela Universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv))

### 1.2. Pandemia Coronavírus 2019

"A COVID-19 é uma infecção respiratória aguda causada pelo coronavírus SARS-CoV-2, potencialmente grave, de elevada transmissibilidade e de distribuição global."

A disponibilidade de dados sobre a evolução de uma pandemia no tempo em determinada região geográfica é fundamental para o seu combate. Com este projeto, foi criado um dashboard de dados para exploração e visualização interativa de dados sobre o avanço de casos e da vacinação no Brasil no ano 2021.

### 1.3. Dados

Os dados sobre casos da COVID-19 são compilados pelo centro de ciência de sistemas e engenharia da Universidade Americana John Hopkins. Os dados foram atualizados diariamente desde Janeiro de 2020 até Março de 2023, com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.). O website do projeto pode ser acessado no neste [link](https://systems.jhu.edu/research/public-health/ncov/). Os dados utilizados para este projeto podem ser encontrados neste diretório do [GitHub](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports).

Abaixo estão descritos os dados derivados do seu preocessamento:

* **date**: data de referência;
* **state**: estado;
* **country**: país;
* **population**: população estimada;
* **confirmed**: número acumulado de infectados;
* **confirmed_1d**: número diário de infectados;
* **confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;
* **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
* **deaths**: número acumulado de mortos;
* **deaths_1d**: número diário de mortos;
* **deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;
* **deaths_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
* **month**: mês de referência;
* **year**: anode referência.

Os dados sobre vacinação da COVID-19 foram compilados pelo projeto Nosso Mundo em Dados (Our World in Data ou OWID) da Universidade Britânica de Oxford disponível no link https://www.ox.ac.uk/. Os dados são atualizados desde Janeiro de 2020 com uma granularidade temporal de dias e geográfica de países. O website do projeto pode ser acessado no link https://ourworldindata.org/coronavirus.

Abaixo estão descritos os dados derivados do seu processamento:

* **date**: data de referência;
* **country**: país;
* **population**: população estimada;
* **total**: número acumulado de doses administradas;
* **one_shot**: número acumulado de pessoas com uma dose;
* **one_shot_perc**: número acumulado relativo de pessoas com uma dose;
* **two_shots**: número acumulado de pessoas com duas doses;
* **two_shots_perc**: número acumulado relativo de pessoas com duas doses;
* **three_shots**: número acumulado de pessoas com três doses;
* **month**: mês de referência;
* **year**: ano de referência.

# 2. Análise Exploratória de Dados

### 2.1. Introdução

Serão utilizados os seguintes pacotes Python para processar os dados brutos em um formato adequado para o painel para exploração interativa de dados: 

In [ ]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

### 2.2. Casos

Para processar os dados de casos da Universidade John Hopkins é necessário iterar um intervalo de tempo, neste caso o ano de 2021, para poder extraí-los:

In [ ]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

start_date = datetime(2021, 1, 1)
end_date = datetime(2021, 12, 31)

De forma interativa, serão selecionadas as colunas de interesse e referentes ao Brasil:

In [ ]:
cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  caso = pd.read_csv(data_source_url, sep=',')

  caso = caso.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  caso = caso.query('Country_Region == "Brazil"').reset_index(drop=True)
  caso['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = caso
    cases_is_empty = False
  else:
    cases = pd.concat([cases, caso], ignore_index=True)

### 2.3. Vacinações

Os dados de vacinação estão compilados em um único arquivo, portanto o arquivo bruto será extraído e as colunas de interesse serão selecionadas:

In [ ]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

vaccines = vaccines.query('location =="Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

### 2.4. Wrangling Casos

Os dados serão manipulados para que possam ser utilizados no dashboard, garantindo uma boa granularidade e qualidade da base de dados. Primeiramente serão alterados os nomes das colunas, para que sejam facilmente identificadas:

In [ ]:
cases = cases.rename(
    columns={
        'Province_State': 'state',
        'Country_Region': 'country'
    }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

São ajustados os nomes dos estados para que possuam os acentos:

In [ ]:
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

Para enriquecer a base de dados, serão criadas novas colunas: chaves temporais e população estimada dos estados.

In [ ]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year'] = cases['date'].apply(lambda date: date.strftime('%Y'))

cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

São criadas também as colunas de número, média móvel (7 dias) e estabilidade (14 dias) de casos e mortes em cada estado:

In [ ]:
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:
  
  if np.isnan(rate):
    return np.NaN
    
  if rate < 0.75:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

    return status

for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = pd.concat([cases_, cases_per_state], ignore_index=True)

cases = cases_
cases_ = None

Para garantir a consistência da base de dados, é feito o type casting das colunas:

In [ ]:
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

Finalmente, as colunas que serão utilizadas são organizadas no dataframe:

In [ ]:
cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

### 2.5. Wrangling Vacinação

Primeiramente são tratados os dados faltantes. O intúito é preencher os buracos com o valor anterior válido mais próximo:

In [ ]:
vaccines = vaccines.fillna(method='ffill')

A base de dados também será filtrada de acordo com a coluna **date**, para garantir que ambas as bases de dados trabalham com o mesmo período de tempo:

In [ ]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

O nome das colunas é alterado para mais facilidade na manipulação dos dados:

In [ ]:
vaccines = vaccines.rename(
    columns={
        'location': 'country',
        'total_vaccinations': 'total',
        'people_vaccinated': 'one_shot',
        'people_fully_vaccinated': 'two_shots',
        'total_boosters': 'three_shots'
    }
)

São criadas colunas com chaves temporais e dados relativos para enriquecer a base de dados:

In [ ]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year'] = vaccines['date'].apply(lambda date: date.strftime('%Y'))

vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

Para garantir a consistência da base de dados, é feito o type casting das colunas:

In [ ]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

Finalmente, as colunas que serão utilizadas são organizadas no dataframe:

In [ ]:
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

### 2.6. Carregamento

Para a persistência dos dados, será feito o download dos dataframes criados para utilização no dashboard:

In [ ]:
cases.to_csv('./covid-cases.csv', sep=',', index=False)

vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)